## Prepare and read data

In [1]:
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import lightgbm as lgb

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error

In [2]:
pd.set_option("max_columns", 300)
pd.set_option("max_rows", 500)

In [3]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

## Feature Engineering

In [4]:
def get_multi_cat_cols(train_df):
    tmp = train_df.iloc[:1000]
    multi_cols = []
    for c in train_df.columns:
        sep_num = tmp[c].astype(str).fillna("").str.contains(";").sum()
        if sep_num > 10:
            multi_cols.append(c)
    return multi_cols

In [5]:
original_cols = train_df.columns.tolist()

In [6]:
multi_cat_cols = get_multi_cat_cols(train_df)

In [7]:
for c in tqdm(multi_cat_cols):
    binarizer = MultiLabelBinarizer()
    train_multi_srs = train_df[c].map(lambda x: x.split(";") if x is not np.nan else [])
    test_multi_srs = test_df[c].map(lambda x: x.split(";") if x is not np.nan else [])
    train_arr = binarizer.fit_transform(train_multi_srs)
    test_arr = binarizer.transform(test_multi_srs)
    feat_cols = [f"{c}_{val}" for val in binarizer.classes_]
    train_feat_df = pd.DataFrame(train_arr, columns=feat_cols)
    test_feat_df = pd.DataFrame(test_arr, columns=feat_cols)
    train_df = pd.concat([train_df, train_feat_df], axis=1)
    test_df = pd.concat([test_df, test_feat_df], axis=1)

In [8]:
other_cols = [
    c for c in list(np.setdiff1d(original_cols, multi_cat_cols)) if c not in ["Salary", "No"]
]

In [9]:
for c in tqdm(other_cols):
    if train_df[c].dtype.name == "float":
        continue
    else:
        train_df[c], uniques = pd.factorize(train_df[c])
        test_df[c] = uniques.get_indexer(test_df[c])

In [28]:
train_df.shape, test_df.shape

((33857, 532), (11259, 531))

In [30]:
train_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in train_df.columns]
test_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in test_df.columns]

In [12]:
use_cols = [c for c in train_df.columns if c not in multi_cat_cols + ["Salary", "No"]]

In [13]:
folds = KFold(n_splits=5, random_state=2020, shuffle=True)
fold_idx_list = [(trn_idx, val_idx) for trn_idx, val_idx in folds.split(train_df, train_df["Salary"])]

### Target Encoding

In [17]:
for c in tqdm(use_cols):
    if train_df[c].nunique() < 5:
        continue
    new_col = "te_" + c
    train_df[new_col] = 0
    test_df[new_col] = 0
    for trn_idx, val_idx in fold_idx_list:
        mean_val = train_df.loc[trn_idx].groupby(c)["Salary"].mean()
        train_df.loc[val_idx, new_col] = train_df.loc[val_idx, c].map(mean_val)
        test_df.loc[:, new_col] += test_df.loc[:, c].map(mean_val) / len(fold_idx_list)

In [19]:
use_cols = [c for c in train_df.columns if c not in multi_cat_cols + ["Salary", "No"]]

## Training

In [20]:
lgb_params = {
            'objective': 'regression',
            "metric": "rmse",
            "verbosity": -1,
            "boosting": "gbdt",
            'learning_rate': 0.05,
            'num_leaves': 128,
            'min_data_in_leaf': 30, 
            'max_depth': 6,
            "bagging_freq": 10,
            "bagging_fraction": 0.4,
            "lambda_l1": 0.5,
            "lambda_l2": 0.5,
            "feature_fraction": 0.7,
            "seed": 2020,
            "num_threads": -1,
            "max_bins": 30
}

In [24]:
importances = pd.DataFrame()
oof_preds = np.zeros(len(train_df))
models = []

for fold_i, (trn_idx, val_idx) in enumerate(fold_idx_list):
    print(f"Fold {fold_i+1}")
    train_dataset = lgb.Dataset(
        train_df.loc[trn_idx, use_cols],
        label = train_df.loc[trn_idx, "Salary"]
    )
    valid_dataset = lgb.Dataset(
        train_df.loc[val_idx, use_cols],
        label = train_df.loc[val_idx, "Salary"]
    )
    model = lgb.train(
                lgb_params,
                train_dataset,
                3000,
                valid_sets = [train_dataset, valid_dataset],
                verbose_eval=100,
                early_stopping_rounds = 100,
                #feval = eval_f1,
                #callbacks = [log_callback],
    )
    imp_df = pd.DataFrame()
    imp_df['feature'] = use_cols
    imp_df['gain'] = model.feature_importance(importance_type="gain")
    importances = pd.concat([importances, imp_df], axis=0, sort=False)
    
    oof_preds[val_idx] = model.predict(train_df.loc[val_idx, use_cols])
    models.append(model)

Fold 1
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 19610.9	valid_1's rmse: 21658.3
[200]	training's rmse: 17873.4	valid_1's rmse: 21255.1
[300]	training's rmse: 16758.1	valid_1's rmse: 21201.7
[400]	training's rmse: 15767.2	valid_1's rmse: 21133.9
[500]	training's rmse: 14963.4	valid_1's rmse: 21165.9
Early stopping, best iteration is:
[408]	training's rmse: 15698.8	valid_1's rmse: 21120.3
Fold 2
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 19700.2	valid_1's rmse: 21651.6
[200]	training's rmse: 18047.9	valid_1's rmse: 21162
[300]	training's rmse: 16843.7	valid_1's rmse: 20978.5
[400]	training's rmse: 15866.3	valid_1's rmse: 20910.1
Early stopping, best iteration is:
[383]	training's rmse: 16010	valid_1's rmse: 20900.2
Fold 3
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 19636.9	valid_1's rmse: 21637.8
[200]	training's rmse: 17921.5	valid_1's rmse: 21232.3
[300]	traini

In [25]:
oof_score = np.sqrt(mean_squared_error(train_df["Salary"], oof_preds))
oof_score

20975.805889678388

In [26]:
importances.groupby("feature")["gain"].mean().sort_values(ascending=False).reset_index()

,feature,gain
0,te_Country,7.393493e+13
1,te_YearsCodingProf,2.333807e+13
2,te_CurrencySymbol,1.356083e+13
3,MilitaryUS,9.636607e+12
4,te_YearsCoding,9.302839e+12
...,...,...
504,PlatformWorkedWith_ESP_8266,0.000000e+00
505,LanguageWorkedWith_Hack,0.000000e+00
506,PlatformDesireNextYear_ESP_8266,0.000000e+00
507,IDE_Light_Table,0.000000e+00


In [32]:
test_pred = np.zeros(len(test_df))

for model in models:
    test_pred += model.predict(test_df[use_cols]) / len(models)

In [33]:
test_pred

array([ 65449.13786739,  93527.42520156,  87189.92305136, ...,
        95189.30052441,  82192.77348821, 111983.89009628])

In [34]:
sub_df = pd.read_csv("../input/submit.csv")

In [39]:
sub_df["Salary"] = test_pred

In [40]:
sub_df.to_csv("../predict/004_baseline_te_20975.csv", index=False)